# Align and crop a face in an image using dlib

In [ ]:
import cv2
import dlib
import numpy as np
import imutils
from imutils.face_utils import shape_to_np

In [ ]:
import dlib
print(dlib.__version__)


In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

In [ ]:
def align_face(image, landmarks, desired_left_eye=(0.35, 0.35), output_size=(224, 224)):
    # Extract the left and right eye coordinates
    left_eye_pts = landmarks[36:42]
    right_eye_pts = landmarks[42:48]

    left_eye_center = left_eye_pts.mean(axis=0).astype("int")
    right_eye_center = right_eye_pts.mean(axis=0).astype("int")

    # Compute the angle between the eyes
    dY = right_eye_center[1] - left_eye_center[1]
    dX = right_eye_center[0] - left_eye_center[0]
    angle = np.degrees(np.arctan2(dY, dX))  # rotation angle

    # Desired distance between eyes
    desired_dist = (1.0 - 2 * desired_left_eye[0]) * output_size[0]
    actual_dist = np.linalg.norm(right_eye_center - left_eye_center)
    scale = desired_dist / actual_dist

    # Compute center between eyes - used as center of rotation
    eyes_center = ((left_eye_center[0] + right_eye_center[0]) // 2,
                   (left_eye_center[1] + right_eye_center[1]) // 2)

    # Get rotation matrix
    M = cv2.getRotationMatrix2D(eyes_center, angle, scale)

    # Adjust translation
    tX = output_size[0] * 0.5
    tY = output_size[1] * desired_left_eye[1]
    M[0, 2] += (tX - eyes_center[0])
    M[1, 2] += (tY - eyes_center[1])

    # Apply affine transformation
    aligned = cv2.warpAffine(image, M, output_size, flags=cv2.INTER_CUBIC)

    return aligned

def detect_align_crop(image_path, save_path="aligned_face.jpg"):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 1)
    if len(rects) == 0:
        print("No face detected.")
        return

    rect = rects[0]  # Take the first face
    shape = predictor(gray, rect)
    landmarks = shape_to_np(shape)

    aligned_face = align_face(image, landmarks)
    cv2.imwrite(save_path, aligned_face)
    print(f"Aligned face saved to {save_path}")

In [ ]:
# Test:
detect_align_crop("test_face.jpg")